# Транзакции клиентов

Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

- InvoiceNo — номер транзакции
- StockCode — код товара
- Description — описание товара
- Quantity — количество единиц товара, добавленных в заказ
- InvoiceDate — дата транзакции 
- UnitPrice — цена за единицу товара
- CustomerID — id клиента
- Country — страна, где проживает клиент

<b>Задачи</b>:
- сколько всего транзакций отменили пользователи?
- отфильтровать и оставить только те заказы, где Quantity > 0
- посчитать число заказов для каждого пользователя из Германии и оставить только тех, кто совершил более N транзакций, где N – 80% процентиль
- отфильтровать из основного датасета записи только по интересующим нас юзерам из Германии
- какой из продуктов добавляли в корзину чаще всего, кроме POST?
- создать колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice
- для каждой транзакции посчитать финальную сумму заказа и выбрать топ-5 транзакций

## Загрузка данных

In [1]:
import pandas as pd

In [2]:
# загрузка файла в zip-архиве в кодировке ISO-8859-1
retail = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-n-laschevskaja/lesson_5/5_data.csv.zip',\
                     encoding='ISO-8859-1', compression='zip')

In [3]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
# кол-во дубликатов в датасете
retail.duplicated().sum()

5268

In [5]:
# удалим дубликаты
retail = retail.drop_duplicates()

## Задачи

1. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). Сколько всего транзакций отменили пользователи? Каждая строка - это отдельная транзакция.

In [6]:
# строки с отмененными транзакциями
retail[retail.InvoiceNo.str.startswith('C')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


In [7]:
# кол-во транзакций
retail[retail.InvoiceNo.str.startswith('C')].shape[0]

9251

2. Теперь отфильтруем данные и оставим в retail только те заказы, где Quantity > 0. Всего 526054 строк.

In [8]:
retail[retail.Quantity > 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


3. Посчитаем число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставим только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишем полученные id пользователей в germany_top (только id).

In [9]:
# на 1 заказ может быть несколько строк - например, заказ получен, заказ в процессе, заказ доставлен 
# Поэтому в агрегации используем функцию 'nunique'
retail.query('Country == "Germany"')\
    .groupby('CustomerID', as_index=False)\
    .agg({'InvoiceNo': 'nunique'})

,CustomerID,InvoiceNo
0,12426.0,1
1,12427.0,5
2,12468.0,3
3,12471.0,49
4,12472.0,13
...,...,...
90,13814.0,3
91,13815.0,5
92,13816.0,4
93,13817.0,2


In [10]:
# 80 процентиль для колонки InvoiceNo
N = retail.query('Country == "Germany"')\
    .groupby('CustomerID', as_index=False)\
    .agg({'InvoiceNo': 'nunique'})\
    .InvoiceNo\
    .quantile(q=0.8)

In [11]:
# 80-й процентиль кол-ва транзакций - это 9 транзакций
N

9.0

In [12]:
# фильтруем только тех, у кого InvoiceNo > 80 процентиля
quantile_80_germany = retail.query('Country == "Germany"')\
    .groupby('CustomerID', as_index=False)\
    .agg({'InvoiceNo': 'nunique'})\
    .query('InvoiceNo > @N')

In [13]:
quantile_80_germany

,CustomerID,InvoiceNo
3,12471.0,49
4,12472.0,13
6,12474.0,30
8,12476.0,20
12,12481.0,11
16,12500.0,13
37,12569.0,35
44,12600.0,11
52,12619.0,13
53,12621.0,23


In [14]:
# записываем отфильтрованные CustomerID в отдельную переменную 
germany_top = quantile_80_germany.CustomerID

In [15]:
germany_top

3     12471.0
4     12472.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64

4. Отфильтруем наблюдения из основного датасета и оставим только записи по интересующим нас юзерам (из germany_top).

In [16]:
top_retail_germany = retail.query('CustomerID in @germany_top')

In [17]:
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


5. Сгруппируем top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? Одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [18]:
# самый часто добавляемый товар в корзину (кроме POST): 22326
top_retail_germany.groupby('StockCode', as_index=False)\
    .agg({'InvoiceNo': 'count'})\
    .query('StockCode != "POST"')\
    .sort_values('InvoiceNo', ascending=False)\
    .head()

,StockCode,InvoiceNo
409,22326,62
452,22423,55
411,22328,45
474,22467,37
316,22077,34


6. Создадим в основном датасете колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [19]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [20]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


7. Для каждой транзакции (InvoiceNo) посчитаем финальную сумму заказа. Выберем топ-5 (InvoiceNo) по сумме заказа в порядке убывания TotalRevenue.

In [21]:
retail.groupby('InvoiceNo', as_index=False)\
    .agg({'Revenue': 'sum'})\
    .sort_values('Revenue', ascending=False)\
    .head()\
    .rename(columns={'Revenue': 'TotalRevenue'})

,InvoiceNo,TotalRevenue
22025,581483,168469.60
2303,541431,77183.60
18776,574941,52940.94
19468,576365,50653.91
9741,556444,38970.00


## Выводы

В ходе работы были получены ответы на вопросы и решены задачи:
- всего было отменено 9251 транзакции
- в датасете 526054 транзакций, где заказанное кол-во товаров больше нуля
- найдены пользователи из Германии, которые совершили более 9 транзакций (80-й процентиль)
- отфильтрованы в основном датасете транзакции только по интересующим нас пользователям из Германии
- самый часто добавляемый товар в корзину (кроме POST): 22326
- создана колонка Revenue с общей суммой покупки
- для каждой транзакции посчитана финальная сумма заказа и выбраны топ-5 транзакций